## C/C++ Arrays in a __firstprivate__ Clause

The following example illustrates the size and value of list items of array or  pointer type in a __firstprivate__ clause . The size of new list items is  based on the type of the corresponding original list item, as determined by the  base language.

In this example:

*  The type of __A__ is array of two arrays of two ints.
*   The type of __B__ is adjusted to pointer to array of __n__  ints, because it is a function parameter.
*   The type of __C__ is adjusted to pointer to int, because  it is a function parameter.
*   The type of __D__ is array of two arrays of two ints.
*   The type of __E__ is array of __n__ arrays of __n__  ints.

Note that  __B__ and __E__ involve variable length array types.

The new items of array type are initialized as if each integer element of the original  array is assigned to the corresponding element of the new array. Those of pointer  type are initialized as if by assignment from the original item to the new item.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: carrays_fpriv.1
* type: C
*/
#include <assert.h>

int A[2][2] = {1, 2, 3, 4};

void f(int n, int B[n][n], int C[])
{
  int D[2][2] = {1, 2, 3, 4};
  int E[n][n];

  assert(n >= 2);
  E[1][1] = 4;

  #pragma omp parallel firstprivate(B, C, D, E)
  {
    assert(sizeof(B) == sizeof(int (*)[n]));
    assert(sizeof(C) == sizeof(int*));
    assert(sizeof(D) == 4 * sizeof(int));
    assert(sizeof(E) == n * n * sizeof(int));

    /* Private B and C have values of original B and C. */
    assert(&B[1][1] == &A[1][1]);
    assert(&C[3] == &A[1][1]);
    assert(D[1][1] == 4);
    assert(E[1][1] == 4);
  }
}

int main() {
  f(2, A, A[0]);
  return 0;
}